# Prompt Techniques

> Using Langchain and OpenAI

In [1]:
#pip install openai

In [2]:
#pip install langchain

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Prompt Templates from LangChain

In [5]:
from langchain import PromptTemplate

demo_template = """I want you to act as a product description assistant for people. In a easy way, explain the basics of {product}."""
prompt = PromptTemplate(
  input_variables=["product"],
  template=demo_template,
)
print(prompt.format(product="mobile phone"))

I want you to act as a product description assistant for people. In a easy way, explain the basics of mobile phone.


### Zero-Shot prompting

In [6]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.7)
chain = LLMChain(llm=llm, prompt=prompt)
chain.run("mobile phone")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nA mobile phone is a small electronic device that allows you to make and receive calls, send and receive text messages, and access the internet. It is also commonly known as a cell phone or smartphone.\n\nMobile phones are designed to be portable and can easily fit in your pocket or purse. They are powered by a battery and can be charged using a charger or via a USB cable connected to a computer or power outlet.\n\nOne of the key features of a mobile phone is its ability to make and receive calls. This is done by connecting to a cellular network, which allows you to communicate with others who have mobile phones or landline phones.\n\nIn addition to calls, mobile phones also allow you to send and receive text messages. These are short written messages that can be sent to other mobile phone users.\n\nAnother important feature of mobile phones is their internet connectivity. This allows you to access the internet, check emails, browse websites, and use various apps.\n\nMost modern mo

In [ ]:
chain.run("earbuds")

'\n\nEarbuds are small, lightweight headphones that fit snugly into your ears. They are designed to provide a convenient and comfortable way to listen to music, podcasts, or other audio on-the-go. \n\nMost earbuds have a small, round shape with a soft silicone or foam tip that fits into the ear canal. This helps to block out external noise and provide a better listening experience. \n\nEarbuds come in both wired and wireless options. Wired earbuds have a cord that connects to your device, while wireless earbuds use Bluetooth technology to connect wirelessly. \n\nOne of the main benefits of earbuds is their portability. They are small enough to fit in your pocket or bag, making them perfect for travel or daily use. \n\nWhen choosing earbuds, it is important to consider factors such as sound quality, comfort, and durability. Some earbuds also come with additional features such as noise-cancellation or sweat-resistance for sports and exercise. \n\nOverall, earbuds are a convenient and pra

In [ ]:
chain.run("laptop")

"\n\nSure, I'd be happy to assist you in understanding the basics of a laptop. A laptop is a portable and compact computer that is designed to be used on the go. It typically consists of a screen, keyboard, touchpad, and speakers all built into a single unit.\n\nThe screen of a laptop can vary in size, but it is usually between 13-15 inches diagonally. It displays information and images and allows you to interact with the laptop through touch or a mouse.\n\nThe keyboard is used for typing and inputting commands into the laptop. It typically includes letters, numbers, and special keys for various functions. Some laptops also have a touchpad, which acts as a mouse and allows you to navigate and click on the screen without using an external mouse.\n\nLaptops also have built-in speakers that allow you to listen to music, watch videos, or participate in video calls.\n\nThe main components of a laptop include the processor, memory, storage, and graphics card. The processor is like the brain 

#### Language Translation Using Zeroshot prompt

In [7]:
demo_template1 = '''In an easy way translate the following sentence '{sentence}' into {target_language}'''
language_prompt = PromptTemplate(
  input_variables=["sentence", 'target_language'],
  template=demo_template1,
)
print(language_prompt.format(sentence="How are you", target_language="Tamil"))

In an easy way translate the following sentence 'How are you' into Tamil


In [8]:
chain1 = LLMChain(llm=llm, prompt=language_prompt)
chain1.run({'sentence': 'Hello How are you', 'target_language': 'Tamil'})

'\n\nஹலோ நீங்கள் எப்படி இருக்கிறீர்கள்? (halō nīṅkaḷ eppaṭi irukkiṟīrkaḷ?)'

### Few-shot Prompting 

In [9]:
from langchain import FewShotPromptTemplate

In [10]:
examples = [
  {
    "word": "dog",
    "translation": "நாய்"
  },
  {
    "word": "cat",
    "translation": "பூனை"
  },
  {
    "word": "mouse",
    "translation": "எலி"
  },
]

example_format = '''Word: {word} Translation: {translation}'''

example_prompt = PromptTemplate(
  input_variables=["word", "translation"],template=example_format,
)

In [11]:
few_shot_prompt = FewShotPromptTemplate(
  examples=examples,
  example_prompt=example_prompt,
  prefix="Give me an easy way to Translate\n",
  suffix="Word: {word}\nTranslation: ",
  input_variables=["input"],
  example_separator="\n"
)

In [12]:
print(few_shot_prompt.format(word="Elephant"))

Give me an easy way to Translate

Word: dog Translation: நாய்
Word: cat Translation: பூனை
Word: mouse Translation: எலி
Word: Elephant
Translation: 


In [13]:
chain2 = LLMChain(llm=llm, prompt=few_shot_prompt)
chain2.run({"word": "lion"})

'சிங்கம்'

### Chain of Thoughts

In [15]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


template ="""
Step1 :

I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
"""

prompt = PromptTemplate(
    input_variables=["input","perfect_factors"],
    template = template
)
llm = OpenAI(temperature=0.7)
chain1 = LLMChain(llm=llm,
    prompt=prompt,
    output_key="solutions"
)

template ="""
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:"""

prompt = PromptTemplate(
    input_variables=["solutions"],
    template = template
)

llm = OpenAI(temperature=0.7)
chain2 = LLMChain(llm=llm,
    prompt=prompt,
    output_key="review"
)

template ="""
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:"""

prompt = PromptTemplate(
    input_variables=["review"],
    template = template
)

llm = OpenAI(temperature=0.7)
chain3 = LLMChain(llm=llm,
    prompt=prompt,
    output_key="deepen_thought_process"
)

template ="""
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:"""

prompt = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template = template
)

llm = OpenAI(temperature=0.7)
chain4 = LLMChain(llm=llm,
    prompt=prompt,
    output_key="ranked_solutions"
)

In [16]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(overall_chain({"input":"human colonization of Mars", "perfect_factors":"The distance between Earth and Mars is very large, making regular resupply difficult"}))


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': ' Implementing stricter laws and regulations on plastic usage and disposal\n\nRank: 1st\n\nJustification: This solution has a high probability of success as it directly addresses the root cause of the ocean plastic pollution problem. By implementing stricter laws and regulations, individuals and companies will be held accountable for their plastic usage and disposal. This would also create a ripple effect, as it would raise awareness and encourage people to reduce their plastic consumption. The confidence level is also medium to high, as many countries have already implemented similar measures with positive results. However, there may be challenges in enforcement and compliance, which may affect the overall success of this solution.\n\nB: Investing in and promot